# Zero shot NER classification experiments - testing and comparing different LLM models

### Introduction

Zoning By-laws contain important information about land use, building height, density, and other development regulations. They are important documents that inform urban planning and development decisions in cities.

They are often stored as long, unstructured PDF legal documents and it's difficult to find information within them. Zoning information is also spatial and tied to geospatial datasets. It would be great if the zoning information in the by-laws could be extracted in an efficient and automated way and joined with geospatial datasets.

### Why DistilBERT and LegalBERT
f

### Why QA models? Comparing different models
| Approach                           | What it does                                                                           | Pros                                                                                                                             | Cons                                                                                                                 |
| ---------------------------------- | -------------------------------------------------------------------------------------- | -------------------------------------------------------------------------------------------------------------------------------- | -------------------------------------------------------------------------------------------------------------------- | ---------------------------------------------------------------- |
| **Text Classification**            | Assigns a label to an entire chunk of text (e.g. "this section contains height rules") | Simple to set up; works well if zoning is neatly sectioned                                                                       | Can’t extract numeric values; only gives category                                                                    |
| **NER (Named Entity Recognition)** | Finds predefined entities in text (e.g. `HEIGHT=9.1 m`, `LOT_COVERAGE=35%`)            | Good for structured outputs; works well if entity spans are clearly defined                                                      | Requires labeled token-level data; zoning text is irregular (tables, bullets, weird formatting); not great zero-shot |
| **QA (Question Answering)**        | Extracts a text span from context given a natural-language question                    | Works very well zero-shot with SQuAD models; doesn’t need special labeling format; flexible (“What is the max building height?”) | Requires splitting long contexts; can hallucinate occasionally                                                       ||


### Imports and Set Up

In [ ]:
from transformers import pipeline
import pandas as pd
import evaluate

In [ ]:
# Load SQuAD metrics
squad_metric = evaluate.load("squad")

# Set up array to store LLM responses
results = []

# Evaluation helper function to prepare inputs for Hugging Face SQuAD metrics
def evaluate_model(res, model):

    # res or results: results dictionary containing the outputs of the LLMs/predictions and ground truth
    # model: "distil_answer" or "legal_answer"

    predictions = []
    references = []

    for r in res:
        predictions.append({
            "id": str(r["doc_id"]),
            "prediction_text": r[model]
        })
        references.append({
            "id": str(r["doc_id"]),
            "answers": {
                "text": [r["ground_truth"]],
                "answer_start": [0]  # dummy value
            }
        })

    # Compute metrics
    return squad_metric.compute(predictions=predictions, references=references)

In [3]:
# Create a small labeled evaluation dataset
dataset = [
    {
        "doc_id": 1,
        "context": """
        Maximum Height:       
        Principal Building 
        12.0 m | 4 storeys,    
        Accessory Buildings 
        4.0 m | 1 storey 
        """,
        "question": "What is the maximum building height for accessory buildings?",
        "ground_truth": "4.0 m"
    },
    {
        "doc_id": 2,
        "context": """
        Rowhouse Maximum Lot Area:
        1-3 units: 280m2
        Small-Scale Multi-Unit Maximum Lot Area:
        1-3 units: -,
        4 units: -,
        5-6 units: -
        """,
        "question": "What is the maximum lot area for 5-6 small-scale multi-units?",
        "ground_truth": "not specified"
    },
    {
        "doc_id": 3,
        "context": """
        A child care facility in the R1 District must:
        (a) be limited to a maximum of 25 children;
        (b) be located on a corner lot;
        (c) comply with the development regulations under section 101.4 for 1 to 3 small-scale multi-unit dwelling units on a lot;
        (d) be located on a lot that does not contain a dwelling unit or any other principal use; and
        (e) comply with all other applicable regulations under this Bylaw.
        """,
        "question": "Where does a child care facility in the R1 district be located?",
        "ground_truth": "be located on a corner lot"
    },
    {
        "doc_id": 4,
        "context": """
        some or all of the following regulations may apply to lots in the R1 District on the Community Heritage Register:
        (a) panhandle lots and other irregularly shaped lots may be permitted subject to the following:
            (i) lots with lane access shall have a minimum panhandle width of 1 m that is clear to a height of 2.5 m; and
            (ii) lots without lane access shall have a minimum panhandle width of 4.5 m that is clear to a height of 2.5 m;
        (b) maximum lot coverage as set out in Section 101.4 may be increased to up to 60%;
        (c) all original architectural appurtenances, such as chimneys, railings, vents, decorative features, or similar, may be excluded from the maximum permitted height of a principal building;
        (d) lot line setbacks for street yards may meet a minimum of 2.0 m;
        (e) the minimum separation between buildings on the same lot as required under Section 101.4 may be reduced;
        """,
        "question": "For lots in the R1 district on the Community Heritage Register, what is the minimum lot line setback for street yards?",
        "ground_truth": "2.0 m"
    },
    {
        "doc_id": 5,
        "context": """
        **Dwelling Type**
        **Rowhouse[ .1]** **Small-Scale Multi-Unit**

        Minimum Lot Width[ .2]

        5 m, except 6.2 m for end unit
        Interior Lot 10 m

        lots

        Corner Lot - Street 8 m 10 m

        Corner Lot - Lane 6.2 m 10 m

        Lot Area[ .3]

        Minimum Lot Area       - 281 m[2]

        Maximum Lot Area 280 m[2]       
        .1 At the time of registration of the subdivision plan to create two or more rowhouse lots, the
        registration of a Section 219 Covenant will be required to ensure that all adjoining rowhouse
        dwellings will be constructed at the same time.

        |Permitted Uses|Col2|
        |---|---|
        |Principal Use|Use-Specific Regulations|
        |Small-Scale Multi-Unit Housing|-|
        |Rowhouse Dwellings|101.5.2|
        |Boarding, Lodging, or Rooming House|101.5.3|
        |Group Home|-|
        |Supportive Housing (Category A)|101.5.4|
        |Child Care Facilities|101.5.6|
        |Accessory Use|Use-Specific Regulations|
        |Boarding Use (up to 2 boarders)|-|
        |Home Occupations|6.8, 6.8A|
        |Urban Agriculture|6.30|
        |Accessory Buildings, Structures, and Uses|101.5.5, 6.6|
        """,
        "question": "What is the minimum lot width for a rowhouse that has a street corner lot?",
        "ground_truth": "Corner Lot - Street 8 m"
    },
    {
        "doc_id": 6,
        "context": """
        The minimum number of dwelling units with at least 3 bedrooms must be provided on a lot as follows:
        |Col1|Total Dwelling Units on a Lot|Col3|
        |---|---|---|
        ||1 to 3 Units|4 to 6 Units|
        |Minimum 3+ Bedroom Units:|1 Unit|2 Units|
        """,
        "question": "What is the total number of dwelling units permitted on a lot for 1 to 3 units?",
        "ground_truth": "Minimum 3+ Bedroom Units: 1 Unit"
    },
    {
        "doc_id": 7,
        "context": """
        101.6 General Regulations 101.6.1 Projections (1) The following features may project into the required minimum separation between buildings on the same lot: (a) steps and stairs; 
        (b) ornamental features, such as arbors, trellises, fish ponds, flag poles, or similar landscape features; 
        (c) terraces, decks, or other similar surfaces that are 1.0 m or less above grade; 
        (d) balconies, covered decks, uncovered decks, canopies, sunshades, or other similar features, including supporting structures, that are greater than 1.0 m above grade up to a maximum of 25 percent of the width of a required separation; 
        (e) belt courses, cornices, gutters, sills, chimneys, bay windows, or other similar features up to the lesser of 0.9 m or 25 percent of the width of a required separation; 
        (f) sunken access areas and window wells as per Section 6.9; 
        (g) outdoor appliances; and 
        (h) eaves up to the lesser of 1.2 m (3.94 ft.) or 25 percent of the width of a required separation.
        (2) Permitted projections into required yards are subject to Section 6.12, except that in the R1 District projections into required rear or side yards are limited to a maximum of 0.5 m where the rear or side yard abuts a lane to provide adequate fire truck clearance. 

        101.6.2 Outdoor Areas (1) An outdoor amenity space with a minimum width of 2.0 m and area of 10.0 m2 must be provided for each primary dwelling unit for its exclusive use and be directly accessible from the primary dwelling unit it serves.  
        
        101.6.3 Access and Fire Safety (1) Dwelling units located more than 45 m from a lot line abutting a street shall contain an automatic sprinkler system. 
        (2) All dwelling units shall have a minimum 1.0 m paved or gravel fire access corridor that: (a) provides direct pedestrian access from the dwelling unit entrance to a lot line abutting a street, or abutting a constructed lane where direct access to a street is not feasible; and 
        (b) is clear of any projections or obstructions to a minimum of 2.5 m in height.
        """,
        "question": "What is the minimum width and area for outdoor amenity space for each primary dwelling unit?",
        "ground_truth": "An outdoor amenity space with a minimum width of 2.0 m and area of 10.0 m2"
    }
]

In [4]:
# Load QA Pipelines for each model
# DistilBERT
distilbert_qa = pipeline(
    "question-answering",
    model = "distilbert-base-uncased-distilled-squad"
)

# LegalBERT
legalbert_qa = pipeline(
    "question-answering",
    model="nlpaueb/legal-bert-small-uncased"
)

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/265M [00:00<?, ?B/s]

c:\Users\jtang\AppData\Local\Programs\Python\Python313\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\jtang\.cache\huggingface\hub\models--distilbert-base-uncased-distilled-squad. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


vocab.txt: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/141M [00:00<?, ?B/s]

Device set to use cpu


In [ ]:
# Run zero shot qa for DistilBERT and LegalBERT to compare

for data in dataset:
    q = data['question']
    ctext = data['context']
    truth = data['ground_truth']

    # DistilBERT
    distil_response = distilbert_qa(question=q, context=ctext)['answer']
    # LegalBERT
    legal_response = legalbert_qa(question=q, context=ctext)['answer']

    results.append({
        "doc_id": data['doc_id'],
        "question": q,
        "ground_truth": truth,
        "distil_answer": distil_response,
        "legal_answer": legal_response
    })

dataframe = pd.DataFrame(results)
dataframe

In [ ]:
# Evaluation and metrics

distil_metrics = evaluate_model(results, "distil_answer")
legal_metrics = evaluate_model(results, "legal_answer")

print("DistilBERT Metrics:", distil_metrics)
print("LegalBERT Metrics:", legal_metrics)